# STA 141B Data & Web Technologies for Data Analysis

### Lecture 16, 3/4/25, Natural language processing


### Announcements 


### Today's topics
- Chunking
    - Noun Phrase Chunking
    - Tag Patterns
    - Developing and Evaluating Chunkers
    - Chinking
- Training Classifier-Based Chunkers
- Cascaded Chunker
- Named Entity Recognition
- Relation Extraction

## The Bag-of-words Model

The one-hot encoding, term frequencies, and TF-IDF scores all ignore word order.

The _bag-of-words model_ assumes that the order of words in a document doesn't matter. Imagine taking the words in each document and dumping them into a bag, where they get all mixed up. Note that in this case "model" means a way of thinking about a problem, not a statistical model.

While the order of words in a document might seem important, the bag-of-words model is surprisingly useful. The bag-of-words model is a good place to start if you want to use statistical methods on language data.

#### N-Grams

So far we have tagged unigrams. But they ignore word order. 

In [ ]:
import re
import nltk
import numpy as np
import pandas as pd

In [ ]:
corpus = ["The cat saw the dog was angry at the other cat.", 
          "The dog saw the cat was angry at the other cat.", 
          "The canary saw the iguana was sad.", 
          "They refuse to permit us to obtain the refuse permit."]

In [ ]:
words = [[word.lower() for word in nltk.word_tokenize(document)] for document in corpus]

In [ ]:
words[0]

In [ ]:
a = [i for i in nltk.bigrams(words[0])]
b = [i for i in nltk.bigrams(words[1])]

In [ ]:
a

In [ ]:
b

In [ ]:
len(a)

In [ ]:
len(words[0])

In [ ]:
len(set(a).intersection(set(b))) / len(set(a)) #bigrams

In [ ]:
len(set(words[0]).intersection(set(words[0]))) / len(set(words[0])) #unigrams

In [ ]:
set(words[0]) == set(words[1])

In [ ]:
d = [i for i in nltk.trigrams(words[0])]
e = [i for i in nltk.trigrams(words[1])]

In [ ]:
len(set(d).intersection(set(e))) / len(set(d)) #bigrams

In [ ]:
c = [i for i in nltk.ngrams(words[0], 4)]
d = [i for i in nltk.ngrams(words[1], 4)]

set(d).intersection(set(c))

In [ ]:
d

In [ ]:
def myTokenizer(doc): 
    words = [word.lower() for word in nltk.word_tokenize(doc)]
    return words

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(tokenizer = myTokenizer)
tfidf = vec.fit_transform(corpus)

In [ ]:
tfidfdf=pd.DataFrame(tfidf.todense().T)
#tfidfdf.set_index(pd.Series([tuple(t) for t in vec.get_feature_names_out()]))

In [ ]:
(tfidf @ tfidf.T).todense()

### Tagging 

In [ ]:
words[2]

In [ ]:
nltk.pos_tag(words[2])

In [ ]:
nltk.pos_tag(['canary', 'saw'])

In [ ]:
words[3]

In [ ]:
nltk.pos_tag(words[3])

In [ ]:
w = words[3].copy()
np.random.shuffle(w)
nltk.pos_tag(w)

Although the some taggers tagged unigrams, they are not all unigram taggers: `nltk.pos_tag` considers the position of the token relative to other tokens. In contrast, `regexp_tagger` (which a set of rules that we establish) is a unigram tagger. 

An n-gram tagger is a generalization of a unigram tagger whose context is the current word together with the part-of-speech tags of the n-1 preceding tokens.

We can train taggers on training data. Some tagged training data is available in `nltk`. 

### Chunking

In [ ]:
import re
import requests
import pandas as pd
import time
import lxml.html as lx
import nltk

In [ ]:
def get_info(country):
    time.sleep(0.2)
    result = requests.get(
        'https://www.cia.gov/the-world-factbook/page-data/countries/' 
                          + country + '/page-data.json')
    result.raise_for_status()
    return result.json()

In [ ]:
data = get_info("burma")

In [ ]:
document = [i for i in data['result']['data']['fields']['nodes'] \
            if i.get('name') == 'Background'][0]['data']
document = "".join([t for t in lx.fromstring(document).xpath('//p/text()')])

In [ ]:
document

In [ ]:
def preprocess(document):
    #document = document.lower()
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

In [ ]:
processed_document = preprocess(document)
sentence = processed_document[0]
sentence

#### Noun Phrase Chunking

We will begin by considering the task of noun phrase chunking, or NP-chunking,
where we search for chunks corresponding to individual noun phrases.

One of the most useful sources of information for NP-chunking is part-of-speech tags. This is one of the motivations for performing part-of-speech tagging in our information extraction system. In order to create an NP-chunker, we will first define a chunk grammar, consisting of rules that indicate how sentences should be chunked.

We will define a simple grammar with a single regular expression rule . This rule says that an NP chunk should be formed whenever the chunker finds an optional determiner (DT) followed by any number of adjectives (JJ) and then a noun (NN). Using this grammar, we create a chunk parser, and test it on our example sentence . The result is a tree, which we can either print, or display graphically .

In [ ]:
grammar = "NP: {<DT>?<JJ>*<NN.*>+}"
cp = nltk.RegexpParser(grammar)

In [ ]:
type(cp)

In [ ]:
result = cp.parse(sentence)
type(result)

In [ ]:
print(result)

In [ ]:
result

#### Tag Patterns

The rules that make up a chunk grammar use tag patterns to describe sequences of
tagged words. A tag pattern is a sequence of part-of-speech tags delimited using angle
brackets, e.g.,`<DT>?<JJ>*<NN>`. Tag patterns are similar to regular expression patterns.

In [ ]:
grammar = "NP: {<DT>?<JJ.*>*<NN.*>+}"
cp = nltk.RegexpParser(grammar)
result = cp.parse(sentence)
result

This will chunk any sequence of tokens beginning with an optional determiner, followed by zero or more adjectives of any type, followed by one or more nouns of any type. 

To find the chunk structure for a given sentence, the RegexpParser chunker begins with
a flat structure in which no tokens are chunked. The chunking rules are applied in turn,
successively updating the chunk structure. Once all of the rules have been invoked, the
resulting chunk structure is returned.

The next example shows a simple chunk grammar consisting of two rules. The first rule
matches an optional determiner or possessive pronoun, zero or more adjectives, then a noun. The second rule matches one or more proper nouns. We also define an example
sentence to be chunked , and run the chunker on this input.

In [ ]:
grammar = r"""
    NP: {<DT|P.*>?<JJ>*<NN>+} # chunk determiner/possessive, adjectives and nouns
    {<NNP>+} # chunk sequences of proper nouns
"""
cp = nltk.RegexpParser(grammar)
example = [("Rapunzel", "NNP"), ("let", "VBD"), ("down", "RP"),
            ("her", "PP$"), ("long", "JJ"), ("golden", "JJ"), 
           ("hair", "NN")]
cp.parse(example)

If a tag pattern matches at overlapping locations, the leftmost match takes precedence.

Sometimes it is easier to define what we want to exclude from a chunk. We can define a chink to be a sequence of tokens that is not included in a chunk.

In [ ]:
sentence

In [ ]:
grammar = r""" NP:
    {<.*>+}        # Chunk everything
    }<CC|.*DT|TO>?<\.|,|VB.*>+<IN|TO>?{  # Chink 
"""
cp = nltk.RegexpParser(grammar)
cp.parse(sentence)

As befits their intermediate status between tagging and parsing, chunk
structures can be represented using either tags or trees. The most widespread file representation
uses IOB tags. In this scheme, each token is tagged with one of three special
chunk tags, I (inside), O (outside), or B (begin).

#### Developing and Evaluating Chunkers
Now you have a taste of what chunking does, but we haven’t explained how to evaluate
chunkers. As usual, this requires a suitably annotated corpus. We begin by looking at
the mechanics of converting IOB format into an NLTK tree, then at how this is done
on a larger scale using a chunked corpus. We will see how to score the accuracy of a
chunker relative to a corpus, then look at some more data-driven ways to search for
NP chunks. Our focus throughout will be on expanding the coverage of a chunker.

In [ ]:
# it works like this 
print(cp.accuracy((result,)))

In [ ]:
18 / len(sentence) # 18 tokens have been correctly classified in terms of IOB

In [ ]:
len(sentence)

Using the corpora module we can load the data `conll2000` that has been tagged
then chunked using the IOB notation. The chunk categories provided in this corpus
are NP, VP, and PP.

In [ ]:
from nltk.corpus import conll2000
conll2000.chunked_sents('train.txt')[120]

In [ ]:
len(conll2000.chunked_sents('train.txt'))

In [ ]:
sum([len(i) for i in conll2000.chunked_sents('train.txt')])

In [ ]:
conll2000.chunked_sents('train.txt', chunk_types=['NP'])[99]

In [ ]:
cp = nltk.RegexpParser("") # we are not providing any grammar!
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
print(cp.accuracy(test_sents))

Now let’s try a naive regular expression chunker that
looks for tags beginning with letters that are characteristic of noun phrase tags (e.g.,
`CD` (cardinal number), `DT`, and `JJ`).

In [ ]:
grammar = r"NP: {<[CDJNP].*>+}"
cp = nltk.RegexpParser(grammar) 
#test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
print(cp.accuracy(test_sents))

We could improve on it
by adopting a more data-driven approach, where we use the training corpus to find the
chunk tag (I, O, or B) that is most likely for each part-of-speech tag. In other words, we
can build a chunker using a unigram tagger. But rather than trying to
determine the correct part-of-speech tag for each word, we are trying to determine the
correct chunk tag, given each word’s part-of-speech tag. 

### Cascaded Chunks

So far, our chunk structures have been relatively flat. Trees consist of tagged tokens,
optionally grouped under a chunk node such as NP. However, it is possible to build
chunk structures of arbitrary depth, simply by creating a multistage chunk grammar.

In [ ]:
grammar = r"""
    NP: {<DT|JJ>*<NN.*>+} # Chunk sequences of DT, JJ, NN (noun phrase)
    PP: {<IN><NP>} # Chunk prepositions followed by NP (prepositional phrase)
    VP: {<VB.*><NP|PP>+} # Chunk verbs and their arguments (verb phrase)
"""
cp = nltk.RegexpParser(grammar)
cp.parse(sentence)

This solution is not perfect, as `NP` are needlessly split. We will refine our grammar. 

In [ ]:
grammar = r"""
    NP: {<DT|JJ|NN.*>+} # Chunk sequences of DT, JJ, NN (noun phrase)
    PP: {<IN><NP>} # Chunk prepositions followed by NP (prepositional phrase)
    VP: {<VB.*><NP|PP>+$} # Chunk verbs and their arguments (verb phrase)
    NP: {<NP|PP><CC>?<NP|PP>}
"""
cp = nltk.RegexpParser(grammar)
cp.parse(sentence)

In [ ]:
cp = nltk.RegexpParser(grammar, loop = 2)
cp.parse(sentence)

Recall: The left side takes precedence when assigning the chunks! 

### Named Entity Recognition
Named entities are definite noun phrases that refer to specific types of individuals, such as organizations,
persons, dates, and so on.

| NAMED ENTITY | EXAMPLE | 
| ---- | ---- |
| ORGANIZATION | Georgia-Pacific Corp., WHO |
| PERSON | Eddy Bonte, President Obama |
| LOCATION | Murray River, Mount Everest |
| DATE | June, 2008-06-29 |
| TIME | two fifty a m, 1:30 p.m. |
| FACILITY | Washington Monument, Stonehenge |
| GEO-POLITICAL ENTITIES | South East Asia, Midlothian |

The goal of a named entity recognition (NER) system is to identify textual mentions
of the named entities. This can be broken down into two subtasks: identifying
the boundaries of the NE, and identifying its type. 

How do we go about identifying named entities? One option would be to look up each
word in an appropriate list of names. However, this is prone to errors caused by the fact that many named entity terms
are ambiguous.

Named entity recognition is a task that is well suited to the type of classifier-based
approach that we saw for noun phrase chunking. In particular, we can build a tagger
that labels each word in a sentence using the IOB format, where chunks are labeled by
their appropriate type.

NLTK provides a classifier that has already been trained to recognize named entities,
accessed with the function `nltk.ne_chunk()`. If we set the parameter `binary=True`,
then named entities are just tagged as `NE`; otherwise, the classifier adds category labels
such as `PERSON`, `ORGANIZATION`, and `GPE`.

In [ ]:
import requests
import lxml.html as lx

In [ ]:
r=requests.get('https://plato.stanford.edu/entries/liberalism-latin-america/')
html=lx.fromstring(r.text)
d=" ".join(html.xpath('//div[@id="aueditable"]//p//text()'))

In [ ]:
d[:100]

In [ ]:
import nltk
import re
def preprocess(document):
    document = re.sub("\s+", " ", document)
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

In [ ]:
document=preprocess(d)

In [ ]:
document[0]

In [ ]:
t=[nltk.ne_chunk(sentence) for sentence in document]

In [ ]:
t[15]

In [ ]:
for sent in t:
    for chunk in sent:
        if hasattr(chunk, "label"):
            print(chunk)

### Relation Extraction

Once named entities have been identified in a text, we then want to extract the relations
that exist between them. We will typically be looking for relations
between specified types of named entity. One way of approaching this task is to initially
look for all triples of the form `(X, α, Y)`, where `X` and `Y` are named entities of the required
types, and `α` is the string of words that intervenes between `X` and `Y`.

In [ ]:
for sent in t:
    for i, chunk in enumerate(sent):
        try: 
            if (hasattr(sent[i], "label") and sent[i].label()=="PERSON" 
                and sent[i+1][1]=='IN' 
                and hasattr(sent[i+2], "label") and sent[i+2].label()=="GPE"): 
                relation = {
                    " ".join([i[0] for i in sent[i].leaves()]): 
                    " ".join([i[0] for i in sent[i+2].leaves()])
                }
                print(relation)
        except: 
            pass 